In [1]:
import google.generativeai as genai
import pandas as pd
from copy import deepcopy


/Users/wangsimin/opt/anaconda3/envs/gemini/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prompt preparation

In [2]:
def remove_non_ascii(sentence):
    return ''.join(char for char in sentence if ord(char) < 128)

In [3]:
with open('prompt/prompt_system_etd.txt', 'r') as fp:
    etd_system = fp.readlines()
    
with open('prompt/prompt_system_ps.txt', 'r') as fp:
    ps_system = fp.readlines()

In [4]:
prompt_parts = []
prompt_parts.append(" ".join(etd_system))
prompt_parts

['Use the following step-by-step instructions to respond to user inputs.\n \n Step 1 - The user will provide you with text in triple quotes (Question: """insert text here"""). Find one sentence from the given text which can indicate the answer to "What is the user expecting to do or achieve?". The sentence only describing the system error or undesired behavior should not be considered. Use triple quotes to cite the found sentence. If the given text does not contain such sentence, write "No sentences found to indicate the user\'s expectation. The answer is NO." and skip Step 2 and Step 3.\n \n Step 2 - If found one sentence from Step 1, \n summarize the sentence into one or more patterns delimited by <pattern>..</pattern> according to the syntax and semantic of the sentence. The pattern could be expressed by some keywords and part-of-speech tags from the found sentence, which can explain why this sentence provides the answer to "What is the user expecting to do or achieve?".\n \n Step 3

In [ ]:
prompt_parts = []
prompt_parts.append(" ".join(ps_system))
prompt_parts

In [5]:
df_issue = pd.read_csv("prompt/issue.csv", encoding = "ISO-8859-1")
# df_issue = pd.read_csv("prompt/issue_wrong.csv")

df_etd_example = pd.read_csv("experiment/random_pattern/ETD/gemini/etd_prompt_random2_with_negno11_gemini.csv")
# df_ps_example = pd.read_csv("experiment/random_pattern/PS/gemini/ps_prompt_random3_with_negno11_gemini.csv")

In [6]:
len(df_issue)

1000

In [7]:
issue_clean = [remove_non_ascii(issue) for issue in df_issue["issue"]]
df_issue["issue"] = issue_clean
df_issue.head()

,issue
0,"Hi, when I am trying to apply class dynamicall..."
1,is it possible to get a RouteConfig matched ag...
2,I have angular running from a .Net Core server...
3,"Hello everyone, I want to do sub menu with sea..."
4,I'm struggling with getting this logic [code]....


In [8]:
df_etd_example = df_etd_example.sample(frac=1)
df_etd_example.reset_index(drop=True, inplace=True)
df_etd_example

,Pattern,issue,output,12_examples,14_examples
0,ETD_WANT_TO,""""""" does angular2 always want to load referenc...","The sentence """""" does angular2 always want to ...",0,0
1,ETD_FOR_EXAMPLE,""""""" guys, can somebody help me w/ chained reso...","The sentence """""" e.g. a components needs resul...",1,1
2,ETD_IMPLEMENT,""""""" Hi Guys, I have deployed Angular6 produtio...","The sentence """""" I have deployed Angular6 prod...",0,0
3,ETD_CAN_QUESTION,""""""" hey - can i get some n00b help. I have the...","The sentence """""" can i get some n00b help. """"""...",0,1
4,ETD_HOPE,""""""" Hey everyone. I have a short question abo...","The sentence """""" I hope you can help. """""" uses...",0,0
5,NaN,""""""" i have an array filter in a subscribe bloc...",No sentences found to indicate the user's expe...,1,0
6,ETD_GOING_TO,""""""" I will be grateful if you could take a loo...","The sentence """""" I am going to create chatbot...",1,0
7,ETD_FOR_EXAMPLE,""""""" Any idea on how to provider a service to t...","The sentence """""" It is funny in some places (s...",0,0
8,ETD_MUST,""""""" Hi all, I am looking for a schedular simil...","The sentence """""" must include timeline view, m...",1,0
9,ETD_HAVE_TO,""""""" Hi guys, what do I have to do to tell TS t...","The sentence """""" what do I have to do to tell ...",0,0


In [ ]:
df_ps_example = df_ps_example.sample(frac=1)
df_ps_example.reset_index(drop=True, inplace=True)
df_ps_example

### for all examples

In [ ]:
chat_output = list(df_etd_randexample["output"].values)
# chat_output.extend(list(df_etd_randexample["output_n"].values))
chat_output

In [ ]:
# chat_issue = list(df_etd_example["issue"].values)
chat_issue = ["input: " + q_str for q_str in list(df_ps_example["issue"].values)]
chat_issue

In [ ]:
# chat_output = list(df_etd_example["output"].values)
chat_output = ["output: " + o_str for o_str in list(df_ps_example["output"].values)]
chat_output

### for random/frequent patterns

In [9]:
df_etd_randexample = df_etd_example[df_etd_example["12_examples"]==1]

chat_issue = ["input: " + q_str for q_str in list(df_etd_randexample["issue"].values)]
# chat_issue.extend(list(df_etd_randexample["issue_n"].values))
chat_issue

['input: """ guys, can somebody help me w/ chained resolvers? What is a proper approach to chain resolves? e.g. a components needs result from resolverA; resolverA builds a query depending on a result from resolverB. """',
 'input: """ i have an array filter in a subscribe block. i can console log a variable in the subscribe above my filter, but i\'m getting undefined errors for the same variable in the filter block. anyone have an idea why? """',
 'input: """ I will be grateful if you could take a look on this topic -  I am going to create chatbot using websockets in Angular2. however I have a problem with ERROR TypeError: Cannot read property \'emit\' of undefined """',
 'input: """ Hi all, I am looking for a schedular similar to fullCalendar for angular but a pure free & opensource (must include timeline view , month, year & day view), any recommendations on it would really be of a greater help.... """',
 'input: """ Hello! If I\'m looking to test a component that gets a service inj

In [10]:
chat_output = ["output: " + o_str for o_str in list(df_etd_randexample["output"].values)]
# chat_output.extend(list(df_etd_randexample["output_n"].values))
chat_output

['output: The sentence """ e.g. a components needs result from resolverA; """ uses an example start with <pattern>e.g. a NNS VBZ</pattern> to indicate that the user expects the result from "resolverA". The answer is YES.',
 "output: No sentences found to indicate the user's expectation.",
 'output: The sentence """  I am going to create chatbot using websockets in Angular2. """ uses <pattern>going to VB</pattern> to indicate that the user expects to "create chatbot using websockets in Angular2". The answer is YES.',
 'output: The sentence """ must include timeline view, month, year & day view """ uses <pattern>must VB</pattern> to indicate that the user expects the calendar to include "timeline view, month, year & day view". The answer is YES.',
 'output: The sentence """ can I just provide a mock for the service itself so that I can focus on the actual component? """ uses <pattern>can I VB</pattern> to indicate that the user expects to "only focus on the actual component". The answer 

In [ ]:
df_ps_randexample = df_ps_example[df_ps_example["14_examples"]==1]

chat_issue = ["input: " + q_str for q_str in list(df_ps_randexample["issue"].values)]
# chat_issue.extend(list(df_ps_randexample["issue_n"].values))
chat_issue

In [ ]:
chat_output = ["output: " + o_str for o_str in list(df_ps_randexample["output"].values)]
# chat_output.extend(list(df_ps_randexample["output_n"].values))
chat_output

## Model running

In [17]:
with open("gemini_token.txt", 'r') as fp:
    gemini_token = fp.readline()

In [18]:
genai.configure(api_key=gemini_token)

In [19]:
# Set up the model
generation_config = {
  "temperature": 0.8,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 300,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Create GenerativeModel object
model = genai.GenerativeModel(
    model_name="gemini-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [14]:
for eg_input, eg_output in zip(chat_issue, chat_output):
    prompt_parts.append(eg_input)
    prompt_parts.append(eg_output)

prompt_parts

['Use the following step-by-step instructions to respond to user inputs.\n \n Step 1 - The user will provide you with text in triple quotes (Question: """insert text here"""). Find one sentence from the given text which can indicate the answer to "What is the user expecting to do or achieve?". The sentence only describing the system error or undesired behavior should not be considered. Use triple quotes to cite the found sentence. If the given text does not contain such sentence, write "No sentences found to indicate the user\'s expectation. The answer is NO." and skip Step 2 and Step 3.\n \n Step 2 - If found one sentence from Step 1, \n summarize the sentence into one or more patterns delimited by <pattern>..</pattern> according to the syntax and semantic of the sentence. The pattern could be expressed by some keywords and part-of-speech tags from the found sentence, which can explain why this sentence provides the answer to "What is the user expecting to do or achieve?".\n \n Step 3

In [ ]:
ans1= []
start_i = 0
for i in range(start_i,len(df_issue),1):
# for i in range(0,10,1):
    issue = df_issue.loc[i]["issue"]
    input_str = "input: \"\"\" {} \"\"\"".format(issue)
    
    prompt = deepcopy(prompt_parts)
    prompt.append(input_str)
    prompt.append("output: ")
    
    response = model.generate_content(prompt)
    ans1.append(response.text)

In [16]:
print(len(ans1))

14


In [30]:
df = pd.DataFrame()
df["answer1"] = ans1

df.head(10)

,answer1
0,"The sentence """""" it is messing up my material ..."
1,"The sentence """""" I'm trying to create a Breadc..."
2,"The sentence """""" What's the best way for the a..."
3,"The sentence """""" I want to do sub menu with se..."
4,"The sentence """""" I wanted to set a default val..."
5,"The sentence """""" how can i achieve to display ..."
6,"The sentence """""" Is there a workaround? """""" us..."
7,No sentences found to indicate the user's expe...
8,"The sentence """""" I would like to import variab..."
9,"The sentence """""" I'm using ng-select in my app..."


In [31]:
df.to_csv("experiment/random_pattern/answer_rand.csv",index = None)

In [33]:
from sklearn import metrics as mt

In [34]:
# PS results 

df_test_random = pd.read_csv("experiment/random_pattern/PS/gemini/ps_rand_withnegno1022_gemini_result(t=0.8).csv")
predictions_ps = df_test_random["y''_PS"]
y_test = df_test_random["y_PS"]
precison_ps = mt.precision_score(y_test, predictions_ps)
recall_ps = mt.recall_score(y_test, predictions_ps)
score_ps = mt.f1_score(y_test, predictions_ps)

print("precision:",round(precison_ps,3),"recall:",round(recall_ps,3),"F1:",round(score_ps,3))

precision: 0.644 recall: 0.948 F1: 0.767
